In [1]:
import os
import re
import sys
import dask.array as da
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
import tarfile
from osgeo import gdal, osr
import matplotlib.pyplot as plt

In [2]:
def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = dataset.ReadAsArray(0, 0, cols, rows)
    if im_data.ndim == 3:
        im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols
    
def array_to_geotiff(array, output_path, geo_transform, projection, band_names=None):
    rows, cols, num_bands = array.shape
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(output_path, cols, rows, num_bands, gdal.GDT_Float32)
    
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    
    for band_num in range(num_bands):
        band = dataset.GetRasterBand(band_num + 1)
        band.WriteArray(array[:, :, band_num])
        band.FlushCache()
        
        if band_names:
            band.SetDescription(band_names[band_num])
    
    dataset = None
    band = None
    return
def pixel_to_coord(gt, x, y):
    x_coord = gt[0] + x * gt[1] + y * gt[2]
    y_coord = gt[3] + x * gt[4] + y * gt[5]
    return x_coord, y_coord

In [4]:
## Research_area: SF
input_tif = "AV320230822t183400_L2A_OE_main_98b13fff_RFL_ORT"
output_tif = "AV320230822t183400_L2A_OE_main_98b13fff_RFL_ORT.tif"
gdal.Warp(output_tif, input_tif, format = 'GTiff')
output_tif = None

In [13]:
## Research_area: Richfield

data_path = "/Users/fji/Desktop/rich_AVIRIS/"

file_name = os.listdir(data_path)
file_name = [x for x in file_name if ".tar.gz" in x and "._" not in x]

for file in file_name:
    input_tif = f"{data_path}{file.split('.')[0]}_RFL_ORT"
    output_tif = f"{data_path}{file.split('.')[0]}_RFL_ORT.tif"
    gdal.Warp(output_tif, input_tif, format = 'GTiff')

In [16]:
im_data, im_Geotrans, im_proj,rows, cols = read_tif("/Users/fji/Desktop/rich_AVIRIS/AV320231010t223751_L2A_OE_main_98b13fff_RFL_ORT.tif")
print(im_data.shape, im_Geotrans, im_proj,rows, cols)

(2364, 3884, 284) (387195.50286180223, 0.5000000000000363, 0.0, 4277315.237112288, 0.0, -0.5000000000000363) PROJCS["WGS 84 / UTM zone 12N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32612"]] 2364 3884


In [10]:
## clip the data
data_path = "/Users/fji/Desktop/2_processed_data/"
data_path2 = "/Users/fji/Desktop/3_paired_EMIT_Planet/"
folders = ["2_San_Francisco", "3_Kanab", "4_Richfield"]
for idx, folder in enumerate(folders[1:2]):
    path = f"{data_path}{folder}/2_AVIRIS3/"
    file_names = os.listdir(path)
    file_names = [x for x in file_names if "v0p1" in x and ".hdr" not in x]
    if folder != "3_Kanab":
        shp = f"{data_path2}{folder}/0_clip_shp/0_first_beginning_clip_shp.shp"
        gdf = gpd.read_file(shp)
        bounds = gdf.bounds
        min_x = bounds["minx"].values[0]
        min_y = bounds["miny"].values[0]
        max_x = bounds["maxx"].values[0]
        max_y = bounds["maxy"].values[0]
        ul_x, ul_y = (min_x, max_y)
        lr_x, lr_y = (max_x, min_y)
        print(ul_x, lr_y, lr_x, ul_y)
        for file in file_names:
            in_tif = f"{path}{file}"
            out_tif = f"{path}{file[:-4]}_clipped.tif"
            gdal.Warp(out_tif, in_tif, format = 'GTiff', outputBounds=(ul_x, lr_y, lr_x, ul_y), cutlineDSName=shp, cropToCutline=True)
    else:
        shp = f"{data_path}{folder}/kanab_research_area/kanab_research_areas.shp"
        gdf = gpd.read_file(shp)
        bounds = gdf.bounds
        min_x = bounds["minx"].values[0]
        min_y = bounds["miny"].values[0]
        max_x = bounds["maxx"].values[0]
        max_y = bounds["maxy"].values[0]
        ul_x, ul_y = (min_x, max_y)
        lr_x, lr_y = (max_x, min_y)
        for file in file_names:
            print(file)
            in_tif = f"{path}{file}"
            out_tif = f"{path}{file[:-5]}_clipped.tif"
            dataset = gdal.Open(in_tif)
            geotransform = dataset.GetGeoTransform()
            ul_x2 = geotransform[0]
            ul_y2 = geotransform[3]
            cols = dataset.RasterXSize
            rows = dataset.RasterYSize
            lr_x2 = ul_x2 + (cols * geotransform[1])
            lr_y2 = ul_y2 + (rows * geotransform[5])
            
            ul_x_new = max(ul_x, ul_x2)
            ul_y_new = min(ul_y, ul_y2)
            lr_x_new = min(lr_x, lr_x2)
            lr_y_new = max(lr_y, lr_y2)
            gdal.Warp(out_tif, in_tif, format = 'GTiff', outputBounds=(ul_x_new, lr_y_new, lr_x_new, ul_y_new))
            print("************************************************************************************************")
                    

AV320230925t180116_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************
AV320230925t181559_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************
AV320230925t171815_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************
AV320230925t173235_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************
AV320230925t174711_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************
AV320230925t182957_L2A_OE_main_98b13fff_RFL_ORT_v0p1
************************************************************************************************


In [13]:
# mosaic flightlines
data_path = "/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/"

file_names = os.listdir(data_path)
file_names = [x for x in file_names if "_clipped.tif" in x and ".aux.xml" not in x]
file_names = [f"{data_path}{x}" for x in file_names]

output_path = f'{data_path}AVIRIS3_RFL_20230925.tif'
options = gdal.WarpOptions(format='GTiff')
mosaic = gdal.Warp(output_path, file_names, options=options)
mosaic = None 

In [12]:
file_names

['/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t174711_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif',
 '/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t180116_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif',
 '/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t182957_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif',
 '/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t171815_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif',
 '/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t181559_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif',
 '/Users/fji/Desktop/2_processed_data/3_Kanab/2_AVIRIS3/AV320230925t173235_L2A_OE_main_98b13fff_RFL_ORT_clipped.tif']